In [1]:
import time
import tweepy
import pandas as pd

## Extracting tweet URLs

### Install snscrape in your terminal using: pip install snscrape
### We have chosen out keyword as #Trump, Start Date: 2020-08-10 and End Date: 2020-08-15

### Type the following command in your terminal to extract data: 
#### snscrape twitter-search "#Trump since:2020-08-10 until:2020-08-15" > trump_tweets.txt


### Twitter Credentials (Update your twitter credentials in the credentials.csv file and load it here)

In [2]:
credentials_df = pd.read_csv('twitter_credentials.csv',header=None,names=['Name','Key'])
credentials_df

,Name,Key
0,consumer_key,CvbMui2D4MuIK5dxzT0mR03s8
1,consumer_secret,YkrRUAfCCi9h0mDzThuzNb5d7CSDjQwb4SwDzQWaErVopt...
2,access_token,895342019668619264-lQ9NRAL8URrYfIpRSS1fbVMcUpk...
3,access_secret,naWmOWyX6XDN8f1khdmh2KodtOIr6DOmEjqipyQyetUgV


In [3]:
consumer_key = credentials_df.loc[credentials_df['Name']=='consumer_key','Key'].iloc[0]
consumer_secret = credentials_df.loc[credentials_df['Name']=='consumer_secret','Key'].iloc[0]
access_token = credentials_df.loc[credentials_df['Name']=='access_token','Key'].iloc[0]
access_token_secret = credentials_df.loc[credentials_df['Name']=='access_secret','Key'].iloc[0]

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Load trump_tweets.txt file which contains tweet URLs

In [5]:
tweet_url = pd.read_csv("trump_tweets.txt", index_col= None, header = None, names = ["links"])
tweet_url.head()

,links
0,https://twitter.com/PhilipCohen5/status/129442...
1,https://twitter.com/rdreynola/status/129442341...
2,https://twitter.com/desderamona/status/1294423...
3,https://twitter.com/brosti/status/129442337854...
4,https://twitter.com/k1_Pilot/status/1294423359...


### Extracting Tweet IDs from URLs

In [6]:
af = lambda x: x["links"].split("/")[-1]
tweet_url['ID'] = tweet_url.apply(af, axis=1)
tweet_url.head()

,links,ID
0,https://twitter.com/PhilipCohen5/status/129442...,1294423503748259840
1,https://twitter.com/rdreynola/status/129442341...,1294423414032207874
2,https://twitter.com/desderamona/status/1294423...,1294423402888011776
3,https://twitter.com/brosti/status/129442337854...,1294423378548461581
4,https://twitter.com/k1_Pilot/status/1294423359...,1294423359485284356


In [7]:
idlist = tweet_url['ID'].tolist()

In [8]:
total_tweets = len(idlist)
batch = (total_tweets - 1) // 50 + 1

In [9]:
total_tweets

27343

### Function for extracting elements from tweets using tweet_ids

In [10]:
def extract_tweets(tweet_ids):
    status = api.statuses_lookup(tweet_ids, tweet_mode= "extended")
    df = pd.DataFrame()
    for tweet in status:
            tweet_element = {"tweet_ID": tweet.id,
                     "username": tweet.user.screen_name,
                     "tweet":tweet.full_text,
                     "date":tweet.created_at}
            df = df.append(tweet_element, ignore_index = True)
    df.to_csv("extracted_tweets_trump.csv", mode="a")

In [11]:
for i in range(batch):
        extract_batch = idlist[i*50:(i+1)*50]
        data = extract_tweets(extract_batch)